In [2]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\feder\Documents\GitHub\relgan\src\jupyter_tests


In [3]:
os.chdir("..")
cwd = os.getcwd()
print(cwd)
assert cwd.endswith("src")

C:\Users\feder\Documents\GitHub\relgan\src


# CR dataset management

In [4]:
import nltk, pandas as pd, numpy as np
from os.path import join
from tqdm import tqdm
from utils.text_process import get_word_list, get_dict, text_to_code
from utils.static_file_manage import write_json

In [5]:
file = [
    # 'Canon PowerShot SD500.txt',
    'Canon S100.txt',
    'Diaper Champ.txt',
    'Hitachi router.txt',
    'ipod.txt',
    'Linksys Router.txt',
    'MicroMP3.txt',
    'Nokia 6600.txt',
    'norton.txt',
    'Apex AD2600 Progressive-scan DVD player.txt',
    'Canon G3.txt',
    'Creative Labs Nomad Jukebox Zen Xtra 40GB.txt',
    'Nokia 6610.txt',
    'Nikon coolpix 4300.txt',
]
data_dir = join(os.getcwd(), "..", "resources", "data", "MovieReviews", "cr")

In [6]:
def produce_line(_line):
    _line = _line.strip()
    _line = _line.lower()
    _token = nltk.word_tokenize(_line)
    return _token


def pro_file(file, pos_list, neg_list):
    with open(file, 'r') as f:
        for item in f:
            item = item.strip()
            f_p = 0
            f_n = 0
            f_p = item.find('[+')
            f_n = item.find('[-')
            if f_p != -1:
                pos = item.find('##') + 2
                line_text = item[pos:]
                line_list = produce_line(line_text)
                l = len(line_list)
                if l <= 1500000000 and l > 1:
                    pos_list.append(line_list)
            # print('===============')
            # print(line_text)
            # print(item)
            # input()
            if f_n != -1:
                pos = item.find('##') + 2
                line_text = item[pos:]
                line_list = produce_line(line_text)
                l = len(line_list)
                if l <= 1500000000000 and l > 1:
                    neg_list.append(line_list)
            # print('===============')
            # print(line_text)
            # print(item)
            # input()
    return pos_list, neg_list

In [7]:
pos_list = []
neg_list = []

for file_i in file:
    pos_list, neg_list = pro_file(os.path.join(data_dir, "Reviews-9-products", file_i), pos_list, neg_list)

In [8]:
df = pd.DataFrame(list(zip(np.concatenate([np.ones(len(pos_list), dtype=int), np.zeros(len(neg_list), dtype=int)]), pos_list + neg_list)), columns=["sentiment", "text"])

#### qua c'è la cosa della lunghezza

In [9]:
df['len'] = df['text'].apply(len)
df = df[df['len'] <= 50]

In [10]:
len(df)

3712

In [11]:
token_list = [x for sent in df['text'] for x in sent]

In [12]:
token_list[:10]

['i', 'want', 'to', 'start', 'off', 'saying', 'that', 'this', 'camera', 'is']

In [13]:
from collections import Counter 
def removeElements(lst, k): 
    counted = Counter(lst) 
    return [el for el in lst if counted[el] >= k] 

token_list_removed = removeElements(token_list, 10) #at least 10 times

In [14]:
word_set = list(dict.fromkeys(set(token_list)))
[word_index_dict, index_word_dict] = get_dict(word_set)
sequence_len = len(max(df['text'], key=len))

In [15]:
def apply_text_to_code(x):
    tokenization = nltk.word_tokenize(x.lower())
    eof_code = len(word_index_dict)  # used to filled in the blank to make up a sentence with seq_len
    index = 0
    code_int = []
    for word in tokenization:
        try:
            code_int.append(int(word_index_dict[word]))
        except KeyError:
            print("sta eliminando qualcosa: {}".format(word))
            return np.nan
        index += 1
    while index < sequence_len:
        code_int.append(int(eof_code))
        index += 1
    return code_int

In [16]:
df['text'] = df['text'].apply(lambda x: " ".join(x))
df['tokenized_text'] = df['text'].apply(lambda x: apply_text_to_code(x))
df.dropna(inplace=True)

In [17]:
df.to_csv(join(data_dir, 'train' + '.csv'))

In [20]:
sequence_len

50

In [21]:
config_dict = {}
config_dict['seq_len'] = sequence_len
config_dict['vocabulary_size'] = len(word_index_dict) + 1
config_dict['sentiment_num'] = max(df['sentiment']) + 1

write_json(join(data_dir, 'word_index_dict.json'), word_index_dict)
write_json(join(data_dir, 'index_word_dict.json'), index_word_dict)
write_json(join(data_dir, 'config.json'), config_dict)

In [22]:
df

,sentiment,text,len,tokenized_text
0,1,i want to start off saying that this camera is...,15,"[3311, 4468, 2993, 4585, 5141, 760, 452, 3568,..."
1,1,"i 'm in high school , and this camera is perfe...",44,"[3311, 2822, 2734, 3014, 4102, 1897, 800, 3568..."
2,1,i 'd say that this camera is perfect .,9,"[3311, 3535, 2084, 452, 3568, 3927, 4870, 5364..."
3,1,"ok , not quite everything ... but this camera ...",25,"[2307, 1897, 5520, 1544, 189, 5580, 3409, 3568..."
4,1,i bought this camera for the same reason many ...,25,"[3311, 1759, 3568, 3927, 2950, 1810, 3275, 67,..."
...,...,...,...,...
3785,0,"on average , indoor shots with a flash are not...",16,"[2114, 3781, 1897, 1932, 2613, 1037, 4486, 142..."
3786,0,"i only have one complaint , and that is the 8m...",14,"[3311, 4040, 2643, 3359, 5447, 1897, 800, 452,..."
3787,0,8mb for a camera like this is a joke .,10,"[4899, 2950, 4486, 3927, 5352, 3568, 4870, 448..."
3788,0,1 no direct firewire to the camera .,8,"[5388, 3263, 1181, 4163, 2993, 1810, 3927, 539..."
